In [1]:
import os
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate


from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")

search_tool = SerpAPIWrapper()

python_tool = PythonREPLTool()

response_schemas = [
    ResponseSchema(name="title", description="Title of the research topic"),
    ResponseSchema(name="summary", description="Short summary in 2-3 lines"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

prompt_template = PromptTemplate(
    template="Give a short summary about: {topic}\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

def structured_summary(topic: str) -> str:
    prompt = prompt_template.format(topic=topic)
    response = llm.invoke(prompt)
    return output_parser.parse(response.content)


tools = [
    Tool(
        name="Web Search",
        func=search_tool.run,
        description="Use this tool to search for recent research using SerpAPI"
    ),
    Tool(
        name="Python REPL",
        func=python_tool.run,
        description="Useful for calculations and logic"
    ),
    Tool(
        name="Structured Summary",
        func=structured_summary,
        description="Summarize a research topic in 2–3 lines with title"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Find the latest research in LLMs and summarize it. Then calculate how many tokens are needed for 3 million words."

result = agent.invoke(query)

print("\n🧠 Final Answer:\n", result)


C:\Users\moham\AppData\Local\Temp\ipykernel_21948\2613609350.py:58: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find the latest research in LLMs, I need to search for recent research papers on the topic.

Action: Web Search
Action Input: "latest research on large language models"
Observation: ['The Llama 3.1 models were released in July 2024, including both a 405 billion and 70 billion parameter model. The most recent version is Llama ...', 'Research has shown that large language models (LLMs) tend to overemphasize information at the beginning and end of a document or ...', 'With advancements in LLMs across industries, Booth faculty are key figures leading broader application of language models, from finance to healthcare.', 'The performance of large language models is increasing exponentially, with potential to autonomously complete complex tasks in days rather than months.', 'As we look to 2025, large language models (LLMs) are at the center of a technological revolution that promises not only to transform industries ...', 'Large Language Mod

Python REPL can execute arbitrary code. Use with caution.


Thought: Now that I have a summary of the latest research in LLMs, I need to calculate how many tokens are needed for 3 million words.

Action: Python REPL
Action Input: `total_words = 3000000; average_words_per_token = 1.3; total_tokens = total_words / average_words_per_token; print(total_tokens)`
Observation: 2307692.3076923075

Thought:I now know the final answer.

Final Answer: The latest research in LLMs is focused on improving their efficiency and capabilities, with notable examples including the Switch Transformer and Meta AI's LLaMA. To process 3 million words, approximately 2,307,692 tokens are needed, assuming an average of 1.3 words per token.

> Finished chain.

🧠 Final Answer:
 {'input': 'Find the latest research in LLMs and summarize it. Then calculate how many tokens are needed for 3 million words.', 'output': "The latest research in LLMs is focused on improving their efficiency and capabilities, with notable examples including the Switch Transformer and Meta AI's LLaMA.